In [ ]:
# 01 get book data
- 목적 : 구글 book api 를 사용하여, 카테고리별 책 제목과 설명 데이터를 확보한다
- 진행 방법 : api를 활용하여 데이터 수집
- 결과 : 카테고리별 구글 도서 검색 결과 수집
    

In [63]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [128]:
import sys
sys.path.append('../')
import os
os.getcwd()
from tqdm import tqdm

In [5]:
from urllib.request import urlopen
import requests
import config

In [8]:
url = "https://www.googleapis.com/books/v1/volumes"

In [36]:
query = '건강 피트니스'

In [142]:
pre_params = {
    'q' : query,
    'key' : config.api_key,
    'maxResults':40,
    'start_index' : 0
}

pre_response = requests.get(url, params = pre_params)

last_index = pre_response.json()['totalItems'] // 40 + 1 # 총 량

book_data = {}
for index in tqdm(range(last_index)):
    params = {
        'q' : query,
        'key' : config.api_key,
        'maxResults':40,
        'start_index' : index
    }
    response = requests.get(url, params = params)
    data = response.json()
    total_items = data['totalItems']
    
    for row in data['items']:
        if row['etag'] not in book_data:
            book_data[row['etag']] = {}
            book_data[row['etag']]['cnt'] = 0
        else:
            book_data[row['etag']]['cnt'] +=1
        book_data[row['etag']]['kind'] = row['kind']
        book_data[row['etag']]['id'] = row['id']
        book_data[row['etag']]['selfLink'] = row['selfLink']
        book_data[row['etag']]['title'] = row['volumeInfo']['title'] if 'title' in row['volumeInfo'] else 'none'
        book_data[row['etag']]['authors'] = row['volumeInfo']['authors'][0] if 'authors' in row['volumeInfo'] else 'none'
        book_data[row['etag']]['publisher'] = row['volumeInfo']['publisher'] if 'publisher' in row['volumeInfo'] else 'none'
        book_data[row['etag']]['publishedDate'] = row['volumeInfo']['publishedDate'] if 'publishedDate' in row['volumeInfo'] else 'none'
        book_data[row['etag']]['description'] = row['volumeInfo']['description'] if 'description' in row['volumeInfo'] else 'none'
        book_data[row['etag']]['printType'] = row['volumeInfo']['printType'] if 'printType' in row['volumeInfo'] else 'none'
        book_data[row['etag']]['categories'] = row['volumeInfo']['categories'][0] if 'categories' in row['volumeInfo'] else 'none'

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:29<00:00,  1.24it/s]


In [64]:
# 전체 카테고리 수집
# todo 
## while "nextPageToken" 를 사용해서, 없을 때 까지 request 를 보내고 그 결과를 모두 모아두는 코드 생성하기

In [87]:
category_list = [
    '건강 피트니스',
    '건강 정신 신체',
    '과학 수학',
    '교육',
    '로맨스',
    '비즈니스 투자',
    '소설 문학',
    '아동 도서',
    '양육 가족',
    '여행',
    '역사',
    '예술 디자인',
    '외국어 학습 교재',
    '요리 음식 와인',
    '자기계발',
    '자서전 전기',
    '정치 사회 과학',
    '종교 영성',
    '컴퓨터 및 기술',
    'SF 및 판타지',  
]

In [156]:
for query in category_list:
    print(query)
    pre_params = {
    'q' : query,
    'key' : config.api_key,
    'maxResults':40,
    'start_index' : 0
    }

    pre_response = requests.get(url, params = pre_params)

    last_index = pre_response.json()['totalItems'] // 40 + 1 # 총 량

    book_data = {}
    for index in tqdm(range(last_index)):
        params = {
            'q' : query,
            'key' : config.api_key,
            'maxResults':40,
            'start_index' : index
        }
        response = requests.get(url, params = params)
        data = response.json()
        total_items = data['totalItems']

        for row in data['items']:
            if row['etag'] not in book_data:
                book_data[row['etag']] = {}
                book_data[row['etag']]['cnt'] = 0
            else:
                book_data[row['etag']]['cnt'] +=1
            book_data[row['etag']]['kind'] = row['kind']
            book_data[row['etag']]['id'] = row['id']
            book_data[row['etag']]['selfLink'] = row['selfLink']
            book_data[row['etag']]['title'] = row['volumeInfo']['title'] if 'title' in row['volumeInfo'] else 'none'
            book_data[row['etag']]['authors'] = row['volumeInfo']['authors'][0] if 'authors' in row['volumeInfo'] else 'none'
            book_data[row['etag']]['publisher'] = row['volumeInfo']['publisher'] if 'publisher' in row['volumeInfo'] else 'none'
            book_data[row['etag']]['publishedDate'] = row['volumeInfo']['publishedDate'] if 'publishedDate' in row['volumeInfo'] else 'none'
            book_data[row['etag']]['description'] = row['volumeInfo']['description'] if 'description' in row['volumeInfo'] else 'none'
            book_data[row['etag']]['printType'] = row['volumeInfo']['printType'] if 'printType' in row['volumeInfo'] else 'none'
            book_data[row['etag']]['categories'] = row['volumeInfo']['categories'][0] if 'categories' in row['volumeInfo'] else 'none'
    
    os.makedirs('../result/', exist_ok=True)
    temp_result = pd.DataFrame(book_data).T.to_csv(f'../result/data_{query}.csv', index = False)

건강 피트니스


100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:30<00:00,  1.23it/s]


건강 정신 신체


100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:26<00:00,  1.15it/s]


과학 수학


...

요리 음식 와인


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:25<00:00,  1.10it/s]


자기계발


 45%|████████████████████████████████████▋                                             | 30/67 [00:24<00:29,  1.24it/s]


KeyError: 'totalItems'